In [ ]:
import numpy as np

KAPPA_MAG = np.arctanh(np.sqrt(0.95)) / 2000 * 2 / np.pi


def get_L(delta_k1, delta_k2):
    """線形演算子Lに対応する対角行列の要素を返す"""
    return 1j * np.array([0.0, delta_k1, delta_k1 + delta_k2], dtype=complex)


def N(B, kappa):
    """非線形項N(B)を計算する (ETDRK4用、今回は直接使用しない)"""
    B1, B2, B3 = B
    return 1j * kappa * np.array([
        np.conj(B1) * B2 + np.conj(B2) * B3,
        B1**2 + 2 * np.conj(B1) * B3,
        3 * B1 * B2
    ])


def A_from_B(B, z, delta_k1, delta_k2):
    """正準変数Bから物理的な振幅Aに変換する"""
    phase_factors = np.exp(-1j *
                           np.array([0, delta_k1 * z, (delta_k1 + delta_k2) * z]))
    return B * phase_factors


def generate_periodic_domains(z_start, z_end, period, kappa_mag):
    """周期的な符号反転ドメイン構造を生成する"""
    domains = []
    if period < 1e-12:
        return domains
    half_period = period / 2.0
    current_z = z_start
    sign_flipper = 1.0
    while current_z + half_period <= z_end:
        domains.append((half_period, sign_flipper * kappa_mag))
        current_z += half_period
        sign_flipper *= -1.0
    return domains

# --- IPM1 Predictor Scheme ---


def phi(omega, h):
    """
    IPM1予測子で使用される積分関数 Φ(Ω, h) を計算する。
    Φ(Ω, h) = (e^(Ωh) - 1) / Ω
    """
    if np.abs(omega) < 1e-9:
        # Ωがゼロに近い場合、テイラー展開を用いてゼロ除算を回避
        # Φ(Ω, h) ≈ h + (h^2*Ω)/2 + ...
        return h + h**2 * omega / 2.0
    else:
        return (np.exp(omega * h) - 1.0) / omega


def predictor_ipm1(B_in, h, kappa_val, L):
    """
    予測子: IPM1 (Interaction Picture Method 1st order) スキームを用いて
    1ドメイン内の時間発展を計算する。
    """
    B1n, B2n, B3n = B_in
    L1, L2, L3 = L

    # 非線形発展項 ΔB_NL を計算するための周波数項 (Ω)
    omega_1_1 = L2 - L1 - L1
    omega_1_2 = L3 - L2 - L1
    omega_2_1 = 2 * L1 - L2
    omega_2_2 = L3 - L1 - L2
    omega_3_1 = L1 + L2 - L3

    # 解析積分式を用いて非線形効果による状態変化ベクトル ΔB_NL を算出
    # この項は線形発展項とは別に加算される
    delta_B_NL1 = 1j * kappa_val * np.exp(L1 * h) * (
        np.conj(B1n) * B2n * phi(omega_1_1, h) +
        np.conj(B2n) * B3n * phi(omega_1_2, h)
    )
    delta_B_NL2 = 1j * kappa_val * np.exp(L2 * h) * (
        B1n**2 * phi(omega_2_1, h) +
        2 * np.conj(B1n) * B3n * phi(omega_2_2, h)
    )
    delta_B_NL3 = 1j * 3 * kappa_val * np.exp(L3 * h) * (
        B1n * B2n * phi(omega_3_1, h)
    )

    delta_B_NL = np.array([delta_B_NL1, delta_B_NL2, delta_B_NL3])

    # 線形発展項を計算
    B_linear = np.exp(L * h) * B_in

    # 線形発展と非線形発展を組み合わせて、ステップ終了時の状態を予測
    B_pred = B_linear + delta_B_NL

    return B_pred


def simulate_superlattice(B0, domains, L):
    """超格子構造全体のシミュレーションを実行する"""
    z = 0.0
    B = B0.astype(complex)

    for h, kappa_val in domains:
        if h < 1e-12:
            continue

        B = predictor_ipm1(B, h, kappa_val, L)

        z += h

    return z, B


def get_scenarios(kappa_mag):
    """シミュレーションシナリオを定義する"""
    DELTA_K1_SHG = 2 * np.pi / 7.2
    DELTA_K2_SFG = 3.2071
    period_shg = 2 * np.pi / DELTA_K1_SHG
    period_higher_order_shg = period_shg * 3
    period_sfg = 2 * np.pi / DELTA_K2_SFG
    Z_MAX_1, Z_MAX_2, Z_MAX_3, Z_SPLIT = 6000.0, 2400.0, 4400.0, 2000.0

    return [
        {
            "name": f"Case 1: QPM for Higher Order SHG (Λ={period_higher_order_shg:.2f} μm)",
            "domains": generate_periodic_domains(0.0, Z_MAX_1, period_higher_order_shg, kappa_mag),
            "A0": np.array([1.0, 0.0, 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        },
        {
            "name": f"Case 2: QPM for SFG (Λ≈{period_sfg:.2f} μm)",
            "domains": generate_periodic_domains(0.0, Z_MAX_2, period_sfg, kappa_mag),
            "A0": np.array([np.sqrt(0.5), np.sqrt(0.5), 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        },
        {
            "name": "Case 3: Cascaded QPM for THG",
            "domains": (
                generate_periodic_domains(0.0, Z_SPLIT, period_shg, kappa_mag) +
                generate_periodic_domains(
                    Z_SPLIT, Z_MAX_3, period_sfg, kappa_mag)
            ),
            "A0": np.array([1.0, 0.0, 0.0]),
            "delta_k1": DELTA_K1_SHG, "delta_k2": DELTA_K2_SFG
        }
    ]


if __name__ == "__main__":
    scenarios = get_scenarios(KAPPA_MAG)

    for config in scenarios:
        print(f"\n--- {config['name']} ---")
        L = get_L(config["delta_k1"], config["delta_k2"])
        initial_I = np.sum(np.abs(config["A0"])**2)

        final_z_corr, final_B_corr = simulate_superlattice(
            config["A0"], config["domains"], L)
        final_A_corr = A_from_B(
            final_B_corr, final_z_corr, config["delta_k1"], config["delta_k2"])
        final_I_corr = np.abs(final_A_corr)**2

        print(f"Final z: {final_z_corr:.2f} μm")
        print(
            f"Final Intensities (I1, I2, I3): {final_I_corr[0]:.8f}, {final_I_corr[1]:.8f}, {final_I_corr[2]:.8f}")
        print(
            f"Total Intensity: {np.sum(final_I_corr):.8f} (Error: {np.sum(final_I_corr) - initial_I:.2e})")


--- Case 1: QPM for Higher Order SHG (Λ=21.60 μm) ---
Final z: 5994.00 μm
Final Intensities (I1, I2, I3): 0.49980856, 0.50137413, 0.00006785
Total Intensity: 1.00125055 (Error: 1.25e-03)

--- Case 2: QPM for SFG (Λ≈1.96 μm) ---
Final z: 2399.96 μm
Final Intensities (I1, I2, I3): 0.24989365, 0.00009198, 0.75093214
Total Intensity: 1.00091777 (Error: 9.18e-04)

--- Case 3: Cascaded QPM for THG ---
Final z: 4397.96 μm
Final Intensities (I1, I2, I3): 0.24898181, 0.00011319, 0.75297929
Total Intensity: 1.00207429 (Error: 2.07e-03)
